In [ ]:
import os
import time

import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import StratifiedKFold, train_test_split

from classification.datasets import Dataset
from classification.utils.audio_student import AudioUtil, Feature_vector_DS
from classification.utils.plots import (
    plot_decision_boundaries,
    plot_specgram,
    show_confusion_matrix,
)
from classification.utils.utils import accuracy

import tensorflow as tf
from keras import layers, models

dataset = Dataset()

fm_dir = "data/feature_matrices/"
model_dir = "data/models/"

nclass = dataset.nclass
naudio = dataset.naudio
classnames = dataset.list_classes()
num_classes = len(classnames)

In [ ]:
for i in range(10):

    print("Iteration: ", i)

    np.random.seed(int(time.time()))

    print("seed = ", int(time.time()))

    myds = Feature_vector_DS(dataset, Nft=512, nmel=20, duration=950, shift_pct=0.8, data_aug=['noise', 'echo', 'time_shift', 'scaling'])

    feature_length = len(myds[classnames[0], 0])
    data_aug_factor = myds.data_aug_factor


    X_aug = np.zeros((data_aug_factor * nclass * naudio, 20, 20))
    y_aug = np.zeros(data_aug_factor * nclass * naudio)

    for s in range(data_aug_factor):
        for idx in range(dataset.naudio):
            for class_idx, classname in enumerate(classnames):
                y_aug[s* nclass * naudio + class_idx * naudio + idx] = class_idx
                featvec = myds[classname, idx]
                X_aug[ s* nclass * naudio + class_idx * naudio + idx, :, :] = featvec.reshape(20, 20)
    np.save(fm_dir + "feature_matrix_2D_aug.npy", X_aug)
    np.save(fm_dir + "labels_aug.npy", y_aug)

    X_aug = np.load(fm_dir + "feature_matrix_2D_aug.npy")
    y_aug = np.load(fm_dir + "labels_aug.npy")

    X_train, X_test, y_train, y_test = train_test_split(
        X_aug, y_aug, test_size=0.2, stratify=y_aug
    )

    """
    model = models.Sequential([
        layers.Input(shape=(20, 20, 1)),
        layers.Normalization(),
        layers.Conv2D(256, kernel_size=(6, 6), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.2),
        layers.Conv2D(512, kernel_size=(6, 6), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(256, activation='leaky_relu'),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adamw',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    model.summary()
    """

    model = models.load_model(model_dir + "one.keras")

    history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

    # store the model

    model.save(model_dir + "one.keras", zipped=True)

    # evaluate the model

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f'Test loss: {test_loss}')

    # plot the confusion matrix
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    show_confusion_matrix(y_pred_classes, y_test, classnames)
    print (f'Test accuracy: {test_acc}')